# **Data cleaning**

## Objectives

**Perform Business requirement 2 user story task: Data cleaning and preparation ML tasks**
* Find and correct if necessary invalid data.
* Handle outliers.
* Split dataset in to train and test subsets.
* Impute missing data.
* Create data cleaning pipeline.




## Inputs
* house prices dataset: outputs/datasets/collection/house_prices.csv

## Outputs


---

## Change working directory

Working directory changed to its parent folder.

In [ ]:
import os
current_dir = os.getcwd()
current_dir

In [ ]:
os.chdir(os.path.dirname(current_dir))
os.getcwd()

---

## Load house price dataset

In [ ]:
import pandas as pd

house_prices_df = pd.read_csv(filepath_or_buffer='outputs/datasets/collection/house_prices.csv')
house_prices_df.dtypes

We know from the data collection notebook, that there are no duplicates in the dataset.

---

## Invalid data

### Data types

Inspection of the data types for each variable, shows no discrepancies from the expectation for each variable's suitable data type.

### Value ranges

Checking the values for each variable are within the numeric valid range or equal to one of the categorical options, as indicated in the datasets metadata.

**First for numeric variables**.

In [ ]:
numeric_house_prices_df = house_prices_df.select_dtypes(exclude=['object'])
numeric_house_prices_df.columns.tolist()

In [ ]:

def check_value_ranges(variable, value_range):
    """
    Checks whether the non-missing values for a 'house_prices_df' numeric variable are in the valid variable range.

    Args:
        variable (str): name of variable.
        value_range (list): [minimum value, maximum value].
    
    Returns a boolean indicating whether all values of the variable are in the valid range.

    """
    variable_series = house_prices_df[variable]
    # drop missing data
    variable_series.dropna(inplace=True)
    result_series = variable_series[variable_series <= value_range[1]]
    result_series = result_series >= value_range[0]
    return result_series.size == variable_series.size


In [ ]:
print('|Variable|Valid range|Data in valid range|')
variable_value_ranges = {'1stFlrSF': [334, 4692], '2ndFlrSF': [0, 2065], 'BedroomAbvGr': [0, 8], 'BsmtFinSF1': [0, 5644],
                         'BsmtUnfSF': [0, 2336], 'TotalBsmtSF': [0, 6110], 'GarageArea': [0, 1418], 'GarageYrBlt': [1900, 2010],
                         'GrLivArea': [334, 5642], 'LotArea': [1300, 215245], 'LotFrontage': [21, 313], 'MasVnrArea': [0, 1600],
                         'EnclosedPorchSF': [0, 286], 'OpenPorchSF': [0, 547], 'OverallCond': [1, 10], 'OverallQual': [1,10],
                         'WoodDeckSF': [0, 736], 'YearBuilt': [1872, 2010], 'YearRemodAdd': [1950, 2010], 'SalePrice': [34900, 755000]}

for variable in numeric_house_prices_df.columns:
    print(f'{variable}|', f'{variable_value_ranges[variable]}|', check_value_ranges(variable, variable_value_ranges[variable]))


All non-missing values are in the valid range for each numeric variable.

**Now for categorical variables**.

In [ ]:
categorical_house_prices_df = house_prices_df.select_dtypes(include=['object'])
categorical_house_prices_df.columns.tolist()

In [ ]:
import numpy as np

# include NaN as a valid value 
result_df = categorical_house_prices_df.isin({'BsmtExposure': ['Gd', 'Av', 'Mn', 'No', 'None', np.nan], 'BsmtFinType1': ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'None', np.nan],
                                  'GarageFinish': ['Fin', 'RFn', 'Unf', 'None', np.nan], 'KitchenQual': ['Ex', 'Gd', 'TA', 'Fa', 'Po', np.nan]})
for col in result_df.columns:
    print(result_df[col].value_counts())

All values are valid for the categorical variables (allowing for missing data).

---

## Outliers

Determined from the significant features EDA notebook that for the most significant continuous numeric features in relation to sale price, there were several instances whose vector components were outliers in at least 50% of the continuous features, thus making them more likely multivariate outliers. What's more it was discovered that the components of these instances corresponded to the extremest outliers for multiple features, supporting the idea of a correlation between the number of features for which an instance's component is an outlier, and the extremity of the outliers.

Outliers for each feature (using the whole dataset) were determined using the IQR method, and the indices of the outliers tracked and counted to determine if the same instance gave rise to outliers for other features. It is common that the dataset is first split into train and test sets before handling outliers, perhaps using a transformer such as winsorize; the idea being to minimise the risk of data leakage. However arguably an outlier in the whole data set (at least the most extreme ones) will still be an outlier in a sample of the distribution (if it is present). Also it could be argued that such values if particularly extreme, and depending on the context of the dataset and business aims, offer no value, and potentially impact the ML algorithms. Therefore for this dataset the outliers will be trimmed from the whole dataset. 

**Adding the outlier related functions from the significant feature EDA notebook.**

In [ ]:
# taken from significant_feature_EDA.ipynb
def locate_single_feature_outliers(feature, df):
    """
    Locates outliers for a feature in a dataframe (containing only numeric features) using the IQR method.

    Args:
        feature (str): the feature name.
        df: dataframe containing the feature.

    Returns a list of indices corresponding to the dataframe indices of the outliers.
    """
    sample = df[feature]
    mean = sample.mean()
    SD = sample.std()
    Q1 = sample.quantile(q=0.25)
    Q3 = sample.quantile(q=0.75)
    IQR = Q3 - Q1
    def return_outliers(instance):
        return instance > IQR*1.5 + Q3 or instance < Q1 - 1.5*IQR
    result = sample.apply(func=return_outliers)
    return result[result == True].index.tolist()

In [ ]:
# taken from significant_feature_EDA.ipynb
def locate_all_feature_outliers(df):
    """
    Amalgamates into a single list, the dataframe (containing only numeric features) indices corresponding to all outliers of features in a dataframe.

    args:
        df: dataframe containing numeric features.

    Returns a list. It contains a series with index corresponding to the index of an outlier, and a column value
    corresponding to the number of times the instance is a common outlier across all features. Also contains
    a value_counts series for the series; finally contains a float for the number of features in the dataframe.
    """
    outlier_indices = []
    for col in df.columns:
        found_ouliers = locate_single_feature_outliers(col, df)
        outlier_indices.extend(found_ouliers)
    index_freq = np.array(outlier_indices)
    index_count = np.unique(index_freq, return_counts=True)
    index_count_series = pd.Series(data=index_count[1], index=index_count[0]).sort_values(ascending=False)
    return [index_count_series, index_count_series.value_counts().sort_values(), df.columns.size]

Rediscovering the outlier instances

In [ ]:
continuous_numeric_features = ['1stFlrSF', '2ndFlrSF', 'BsmtFinSF1', 'GarageArea', 'GrLivArea',
                               'LotArea',
                               'LotFrontage',
                               'MasVnrArea',
                               'OpenPorchSF',
                               'TotalBsmtSF']
outlier_series, outlier_series_unique_count, total_feature_num = locate_all_feature_outliers(house_prices_df[continuous_numeric_features])
print('\n','Instances whose component values correspond to potential outliers in more than 50% of continuous numeric features:')
house_prices_df[continuous_numeric_features].loc[outlier_series[outlier_series > 5].index.tolist()]   

**Removing the instances from the whole dataset**

In [ ]:
house_prices_df.drop(labels=house_prices_df.loc[outlier_series[outlier_series > 5].index.tolist()].index.tolist(), inplace=True)

---